In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config
from transformers import get_linear_schedule_with_warmup

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split, RandomSampler, SequentialSampler

import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
# model_name: ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
model_name = "gpt2-medium"
model_save_path = './model'

In [ ]:
configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)

tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

input_sequence = "beef, salt, pepper"
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

model = model.to(device)
#combine both sampling techniques
sample_outputs = model.generate(input_ids.to(device),
                              do_sample = True, max_length = 120,
                              top_k = 50, top_p = 0.85,
                              num_return_sequences = 3)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
!pip install opendatasets


In [5]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews")

Dataset URL: https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews


100%|██████████| 723M/723M [00:41<00:00, 18.2MB/s]


In [7]:
df_recipes = pd.read_csv('/content/foodcom-recipes-and-reviews/recipes.csv')
df_recipes.reset_index(drop=True, inplace=True)



In [10]:
df_recipes.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')

In [11]:
def form_string(ingredient,instruction):
    s = f"<|startoftext|>Ingredients: {ingredient.strip()}. " \
        f"Instructions: {instruction.strip()}<|endoftext|>"
    return s

data = df_recipes.apply(lambda x:form_string(
    x['RecipeIngredientParts'], x['RecipeInstructions']), axis=1).to_list()
data[0]

'<|startoftext|>Ingredients: c("blueberries", "granulated sugar", "vanilla yogurt", "lemon juice"). Instructions: c("Toss 2 cups berries with sugar.", "Let stand for 45 minutes, stirring occasionally.", "Transfer berry-sugar mixture to food processor.", "Add yogurt and process until smooth.", "Strain through fine sieve. Pour into baking pan (or transfer to ice cream maker and process according to manufacturers\' directions). Freeze uncovered until edges are solid but centre is soft.  Transfer to processor and blend until smooth again.", "Return to pan and freeze until edges are solid.", "Transfer to processor and blend until smooth again.", \n"Fold in remaining 2 cups of blueberries.", "Pour into plastic mold and freeze overnight. Let soften slightly to serve.")<|endoftext|>'

In [12]:
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )


In [13]:
batch_size = 2
max_length = 180

# standard PyTorch approach of loading data in using a Dataset class.
class RecipeDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for recipe in data:
            encodings = tokenizer.encode_plus(recipe,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=max_length,
                                              # return a PyTorch tensor
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))


    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = RecipeDataset(data, tokenizer)
print(f"input_ids: {dataset[0][0]} attn_masks: {dataset[0][1]}")

input_ids: tensor([50257, 41222,    25,   269,  7203, 17585, 20853,  1600,   366, 46324,
         4817,  7543,  1600,   366, 10438,  5049, 32132,  1600,   366,   293,
         2144, 13135, 11074, 27759,    25,   269,  7203,    51,   793,   362,
        14180, 36322,   351,  7543, 33283,   366,  5756,  1302,   329,  4153,
         2431,    11, 26547, 10491, 33283,   366, 43260,   275,  6996,    12,
           82, 35652, 11710,   284,  2057, 12649, 33283,   366,  4550, 32132,
          290,  1429,  1566,  7209, 33283,   366,  1273,  3201,   832,  3734,
          264, 12311,    13, 39128,   656, 16871,  3425,   357,   273,  4351,
          284,  4771,  8566, 16009,   290,  1429,  1864,   284, 11372,     6,
        11678,   737, 34917, 18838,  1566, 13015,   389,  4735,   475,  7372,
          318,  2705,    13,   220, 20558,   284, 12649,   290, 13516,  1566,
         7209,   757, 33283,   366, 13615,   284,  3425,   290, 16611,  1566,
        13015,   389,  4735, 33283,   366, 43260,   2

In [14]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create the DataLoaders for our training and validation datasets.
# Get training samples in random order.
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size # Trains with this batch size.
        )

# Get valiation samples sequentially.
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size # Evaluate with this batch size.
        )

In [15]:
configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)
model = model.to(device)
model.resize_token_embeddings(len(tokenizer))

epochs = 3
learning_rate = 2e-5
warmup_steps = 1e2
# to prevent any division by zero in the implementation
epsilon = 1e-8
optim = AdamW(model.parameters(), lr = learning_rate, eps = epsilon)

total_steps = len(train_dataloader) * epochs  # [no batches] x [no epochs]

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)


In [17]:
def infer(prompt):
    input = f"<|startoftext|>Ingredients: {prompt.strip()}"
    input = tokenizer(input, return_tensors="pt")
    input_ids      = input["input_ids"]
    attention_mask = input["attention_mask"]

    output = model.generate(input_ids.to(device),
                            attention_mask=attention_mask.to(device),
                            max_new_tokens=max_length,
                            do_sample = True, top_k = 50, top_p = 0.85)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output


In [18]:
for epoch_i in range(0, epochs):
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels    = batch[0].to(device)
        b_masks     = batch[1].to(device)

        model.zero_grad()
        outputs = model( input_ids = b_input_ids, labels = b_labels,
                         attention_mask = b_masks, token_type_ids = None )

        loss = outputs[0]

        # Get sample every x batches.
        if step % 100 == 0 and not step == 0:
            model.eval()
            print(infer("eggs, flour, butter, sugar"))
            model.train()

        loss.backward()
        optim.step()
        scheduler.step()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: eggs, flour, butter, sugar, salt, egg white, baking soda, cinnamon, cloves, nutmeg, nutmeg extract, pepper, baking powder, flour, vanilla extract, salt, egg, milk, baking powder Instructions: mix eggs, flour, butter, sugar, salt, egg white, baking soda, cinnamon, cloves, nutmeg, nut, nutmeg extract, pepper, baking powder, flour, vanilla extract, salt, egg, milk, baking powder Instructions: mix eggs, flour, butter, sugar, salt, egg white, baking soda, cinnamon, cloves, nutmeg, nutmeg extract, pepper, baking powder, flour, vanilla extract, salt, egg,, milk, baking powder Instructions: mix eggs, flour, butter, sugar, salt, egg white, baking, cinnamon, cloves, nutmeg, nutmeg extract, pepper, baking powder, flour, vanilla extract,


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: eggs, flour, butter, sugar, cocoa powder, vanilla extract, baking soda, cornstarch, milk, lemon juice, vanilla extract, baking soda, salt, baking powder Instructions: Place all ingredients in a medium bowl and whisk until smooth and combined. Fold in the milk and vanilla. Instructions: Beat the eggs until stiff peaks form. Fold in the cornstarch and milk. Add the mixture to a medium sized bowl. Beat the remaining 2/3 of the milk mixture, until the batter is thickened and slightly fluffy, scraping down the sides and back of the bowl occasionally. Fold in the flour and continue to beat until incorporated. Fold in the remaining 1/3 of the milk mixture and the remaining flour mixture. Repeat with remaining flour mixture. Bake at 350 degrees for 12 minutes. Allow to cool. Place in a heavy pan and bake for 5 minutes or until golden brown, removing from pan and using a tooth


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: eggs, flour, butter, sugar, egg substitute, and vanilla extract", "milk", "vanilla extract", "walnuts", "granulated sugar"). Instructions: c("Melt butter and sugar in a large skillet over medium heat.", "Add the flour and stir to combine.", "Add eggs, 1/2 cup of the egg substitute, and vanilla.  Stir constantly.", "When the eggs are fully cooked, add milk and mix well.", "Stir in the walnuts.", "Cook over low heat for about 4 minutes or until eggs are slightly cooked.", "Remove from heat.", "Fold in nuts, granulated sugar, and vanilla.", "Pour into a greased 9x13\" baking pan.", "Bake at 350F for 15-18 minutes.")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: eggs, flour, butter, sugar and salt; stir in milk and vanilla until well blended.", "In a large bowl, beat eggs with a mixer on medium speed until well blended.", "Add flour, mix on low speed until smooth.", "Slowly add sugar, beating well after each addition.", "Stir in melted chocolate chips.", "Pour over chocolate chips and stir in vanilla.", "Spread evenly on ungreased baking sheets.", "Bake at 375 degrees F for 15-20 minutes.", "Cool slightly and frost.")


KeyboardInterrupt: 

In [19]:
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.json',
 './model/merges.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

In [20]:
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_save_path)
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50260, bias=False)
)

In [21]:
infer("eggs, mushroom, butter, sugar")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Ingredients: eggs, mushroom, butter, sugar, salt, pepper, and cayenne pepper. Instructions: c("Heat a large, heavy skillet over medium-high heat. Add butter. Cook, stirring, until butter melts.", "Add half the onion mixture and cook, stirring, until onions are translucent. Stir in sugar and salt.", "Add remaining onion mixture and cook for 5 minutes. Turn heat down to low and cook until sugar and salt are melted. Cool.", "Place the eggs in a bowl and cover with hot water. Cook until lightly set, stirring occasionally. Cook for 1-2 minutes more, stirring occasionally.", \n"Remove egg yolks from the heat and set aside.", "Cook the rest of the ingredients in a bowl on medium-low heat, stirring occasionally.  Cook until eggs are just starting to set, about 3-5 minutes, stirring occasionally.  The mixture will be slightly thickened'

In [23]:
infer("tomato,sugar,potato")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Ingredients: tomato,sugar,potato starch,eggs", "olive oil", "onions", "carrots", "bell peppers", "chicken broth", "water", "fresh chives", "fresh rosemary"). Instructions: c("Bring a large pot of water to a boil.", "Add all the ingredients and simmer for about an hour. If you want the soup to be a bit thinner, simmer for at least an hour or two.", "When the water is ready to boil add the chives and salt and pepper to taste.")'

In [24]:
infer("Pizza Bread,Tomato ketchup,Onion,Cheese")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Ingredients: Pizza Bread,Tomato ketchup,Onion,Cheese,Velveeta cheese,Worcestershire sauce,Dijon mustard,Worcestershire sauce",American cheese",sugar",fresh ground black pepper"). Instructions: c("Brown the pork, using a pork mallet or mallet with 2 inches of butter or oil, to desired doneness.  Add tomato ketchup and onion.  Mix well.  Heat over medium heat for 5 minutes.", "Add the rest of the ingredients, mixing well, then cook about 5 minutes.", "Enjoy!")'